In [ ]:
!pip install ultralytics 
!pip install cvzone 

In [4]:
from ultralytics import YOLO
import cv2 
import cvzone
import numpy 

In [5]:
model = YOLO("YoloWeights/yolov8m.pt")

In [ ]:
# cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")

# while True: 
#     success, img = cap.read()
#     results = model(img,stream=True)
#     for r in results:
#         boxes = r.boxes 
#         for box in boxes:
#             x1, y1, x2 ,y2 = box.xyxy[0]
#             x1, y1, x2 ,y2 = int(x1) ,int(y1) , int(x2), int(y2)
#             w , h = x2-x1,y2-y1
#             cvzone.cornerRect(img,(x1,y1,w,h))
#             conf = math.ceil((box.conf[0]* 100)) / 100 
#             cvzone.putTextRect(img,f'{conf}',(max(0,x1),max(35,y1)))
            
#     cv2.imshow("Image",img)
#     cv2.waitKey(1)
#     cv2.destroyAllWindows()


In [ ]:
# FIRST ITERATION: 


import cv2
import cvzone
import math

cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")

while True:
    success, img = cap.read()
    
    if not success:
        print("End of video or cannot capture the frame.")
        break
    
    results = model(img, stream=True)
    
    for r in results:
        boxes = r.boxes 
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            
            cvzone.cornerRect(img, (x1, y1, w, h))
            
            conf = math.ceil((box.conf[0] * 100)) / 100
            cvzone.putTextRect(img, f'{conf}', (max(0, x1), max(35, y1)))
    
    cv2.imshow("Image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [ ]:
# import tensorflow as tf
# tf.test.if_gpu_availabe()

In [ ]:
# import numpy
# print(numpy.__version__)

In [ ]:
#Second Iteration

import cv2
import cvzone
import math

coco_labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", 
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", 
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", 
    "couch", "potted plant", "bed", "dining table", "toilet", "Computer", "laptop", "mouse", "remote", 
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", 
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")
overlap_threshold = 0.5

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

while True:
    success, img = cap.read()
    if not success:
        print("End of video or cannot capture the frame.")
        break
    
    results = model(img, stream=True)
    person_boxes = []
    laptop_boxes = []
    
    for r in results:
        boxes = r.boxes 
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            class_idx = int(box.cls[0])
            class_name = coco_labels[class_idx] if class_idx < len(coco_labels) else "Unknown"
            
            if class_name == 'person':
                person_boxes.append((x1, y1, x2, y2))
            elif class_name == 'laptop':
                laptop_boxes.append((x1, y1, x2, y2))
            
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            label = f'{class_name} {conf}'
            cvzone.putTextRect(img, label, (max(0, x1), max(35, y1 - 10)), scale=1.5)
    
    overlap_flag = 0
    for person_box in person_boxes:
        for laptop_box in laptop_boxes:
            iou = calculate_iou(person_box, laptop_box)
            if iou > overlap_threshold:
                overlap_flag = 1
                cv2.rectangle(img, (person_box[0], person_box[1]), (person_box[2], person_box[3]), (0, 0, 255), 2)
                cv2.rectangle(img, (laptop_box[0], laptop_box[1]), (laptop_box[2], laptop_box[3]), (0, 0, 255), 2)
                cvzone.putTextRect(img, "Overlap Detected", (50, 50), colorR=(0, 0, 255))
                break

    cv2.imshow("Image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [10]:
#Third iteration

import cv2
import cvzone
import math

coco_labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", 
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", 
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", 
    "couch", "potted plant", "bed", "dining table", "toilet", "Computer", "laptop", "mouse", "remote", 
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", 
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")
overlap_threshold = 0.2

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

while True:
    success, img = cap.read()
    if not success:
        print("End of video or cannot capture the frame.")
        break
    
    results = model(img, stream=True)
    person_boxes = []
    chair_boxes = []

    for r in results:
        boxes = r.boxes 
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            class_idx = int(box.cls[0])
            class_name = coco_labels[class_idx] if class_idx < len(coco_labels) else "Unknown"
            
            if class_name == 'person':
                person_boxes.append((x1, y1, x2, y2))
            elif class_name == 'chair':
                chair_boxes.append((x1, y1, x2, y2))
            
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            label = f'{class_name} {conf}'
            cvzone.putTextRect(img, label, (max(0, x1), max(35, y1 - 10)


SyntaxError: incomplete input (224923195.py, line 61)

In [ ]:
#Fourth iteration 


import cv2
import cvzone
import math
import io
from contextlib import redirect_stdout

coco_labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", 
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", 
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", 
    "couch", "potted plant", "bed", "dining table", "toilet", "Computer", "laptop", "mouse", "remote", 
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", 
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

chair_index = coco_labels.index("chair")
cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")
overlap_threshold = 0.2

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

while True:
    success, img = cap.read()
    if not success:
        break

    with io.StringIO() as buf, redirect_stdout(buf):
        results = model(img, stream=True)

    person_boxes = []
    chair_boxes = []

    for r in results:
        boxes = r.boxes 
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            class_idx = int(box.cls[0])
            class_name = coco_labels[class_idx] if class_idx < len(coco_labels) else "Unknown"
            
            if class_name == 'person':
                person_boxes.append((x1, y1, x2, y2))
            elif class_name == 'chair':
                chair_boxes.append((x1, y1, x2, y2))
            
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            label = f'{class_name} {conf}'
            cvzone.putTextRect(img, label, (max(0, x1), max(35, y1 - 10)), scale=1.5)

    overlap_flag = False
    for person_box in person_boxes:
        for chair_box in chair_boxes:
            iou = calculate_iou(person_box, chair_box)
            if iou > overlap_threshold:
                overlap_flag = True
                cv2.rectangle(img, (person_box[0], person_box[1]), (person_box[2], person_box[3]), (0, 0, 255), 2)
                cv2.rectangle(img, (chair_box[0], chair_box[1]), (chair_box[2], chair_box[3]), (0, 0, 255), 2)
                break

    if overlap_flag:
        print(chair_index)

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Fifth iteration

#all the libraries 
import cv2
import cvzone
import math
import io
from contextlib import redirect_stdout
import time

coco_labels = {}
coco_class_labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", 
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", 
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", 
    "couch", "potted plant", "bed", "dining table", "toilet", "computer", "laptop", "mouse", 
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", 
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]
overlap_timers = {}
cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")
overlap_threshold = 0.2

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area != 0 else 0

while True:
    success, img = cap.read()
    if not success:
        break

    coco_labels.clear()
    with io.StringIO() as buf, redirect_stdout(buf):
        results = model(img, stream=True)
    
    for r in results:
        boxes = r.boxes 
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            class_idx = int(box.cls[0])
            class_name = coco_class_labels[class_idx] if class_idx < len(coco_class_labels) else "Unknown"
            
            if class_name not in coco_labels:
                coco_labels[class_name] = []
            coco_labels[class_name].append((x1, y1, x2, y2))
            
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            label = f'{class_name} {conf}'
            cvzone.putTextRect(img, label, (max(0, x1), max(35, y1 - 10)), scale=1.5)

    current_time = time.time()
    for obj1_name, obj1_boxes in coco_labels.items():
        if obj1_name == "person":
            for obj2_name, obj2_boxes in coco_labels.items():
                if obj1_name != obj2_name:
                    for box1 in obj1_boxes:
                        for box2 in obj2_boxes:
                            iou = calculate_iou(box1, box2)
                            if iou > overlap_threshold:
                                overlap_key = (obj1_name, obj2_name)
                                if overlap_key not in overlap_timers:
                                    overlap_timers[overlap_key] = current_time
                                elif current_time - overlap_timers[overlap_key] > 0:
                                    print(f"{coco_class_labels.index(obj2_name)}")
                                    overlap_timers[overlap_key] = current_time

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#final 

import cv2
import cvzone
import math
import io
from contextlib import redirect_stdout

# COCO class labels
coco_labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", 
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", 
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", 
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", 
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", 
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", 
    "couch", "potted plant", "bed", "dining table", "toilet", "Computer", "laptop", "mouse", "remote", 
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", 
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# Get the index of "chair" in the coco_labels array
chair_index = coco_labels.index("chair")

# Open video file
cap = cv2.VideoCapture("WhatsApp Video 2024-10-08 .mp4")

# Set overlap threshold
overlap_threshold = 0.2

def calculate_iou(box1, box2):
    # Calculate intersection-over-union (IoU) between two boxes
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Calculate the area of intersection
    inter_area = max(0, x2_inter - x1_inter) * max(0, y2_inter - y1_inter)
    
    # Calculate areas of each box and union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    
    # Return IoU value
    return inter_area / union_area if union_area != 0 else 0

while True:
    # Read frame from video
    success, img = cap.read()
    
    # Check if the frame was successfully captured
    if not success:
        break  # End the loop if video ends or fails to capture

    # Suppress output during inference
    with io.StringIO() as buf, redirect_stdout(buf):
        results = model(img, stream=True)
    
    # Initialize lists to store person and chair boxes
    person_boxes = []
    chair_boxes = []

    # Iterate over detection results
    for r in results:
        boxes = r.boxes 
        for box in boxes:
            # Extract bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            
            # Get class index and name
            class_idx = int(box.cls[0])
            class_name = coco_labels[class_idx] if class_idx < len(coco_labels) else "Unknown"
            
            # Append to lists based on class
            if class_name == 'person':
                person_boxes.append((x1, y1, x2, y2))
            elif class_name == 'chair':
                chair_boxes.append((x1, y1, x2, y2))
            
            # Draw bounding box with corners
            cvzone.cornerRect(img, (x1, y1, w, h))
            
            # Display class label and confidence score
            conf = math.ceil((box.conf[0] * 100)) / 100
            label = f'{class_name} {conf}'
            cvzone.putTextRect(img, label, (max(0, x1), max(35, y1 - 10)), scale=1.5)

    # Check for overlap between person and chair boxes
    overlap_flag = False
    for person_box in person_boxes:
        for chair_box in chair_boxes:
            iou = calculate_iou(person_box, chair_box)
            if iou > overlap_threshold:
                overlap_flag = True
                # Draw a red rectangle around the overlapping area
                cv2.rectangle(img, (person_box[0], person_box[1]), (person_box[2], person_box[3]), (0, 0, 255), 2)
                cv2.rectangle(img, (chair_box[0], chair_box[1]), (chair_box[2], chair_box[3]), (0, 0, 255), 2)
                cvzone.putTextRect(img, f"Chair Index: {chair_index}", (50, 50), colorR=(0, 0, 255))
                break

    # Print the index of "chair" in the terminal if any overlap is found
    if overlap_flag:
        print(chair_index)

    # Show the frame with annotations
    cv2.imshow("Image", img)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()
